In [ ]:
!pip install jiwer
!pip install python_speech_features

In [ ]:
import string, math, jiwer
from Lfbank_model import brnn_ctc_lfbank
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from Logfbank_generator import DataGenerator, pad_wav, pad_text, get_wavs_path, get_text_path, dictionary

In [ ]:
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
import os
#print(device_lib.list_local_devices())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth=True

In [ ]:
# variable field
train_path = "/ASR/data/TRAIN"
test_path = "/ASR/data/TEST"
weight_save_path = "/content/drive/My Drive/2020 Fall/NLP/Lfbank/lfbank_Adam_weights2.h5"

nfilt = 80
nfeature = 80
nclass = 27
epoch = 200
nbatch = 32
lr_rate = 10**(-4)
momentum = 0.9

In [ ]:
# get train data
wavs_path = get_wavs_path(train_path)
txts_path = get_text_path(wavs_path)

audios, input_length = pad_wav(wavs_path, nfilt)
texts, label_length = pad_text(txts_path)

In [ ]:
# get test data
wavs_path_val = get_wavs_path(test_path)
txts_path_val = get_text_path(wavs_path_val)

audios_val, input_length_val = pad_wav(wavs_path_val, nfilt)
texts_val, label_length_val = pad_text(txts_path_val)

In [ ]:
# Generators
training_generator = DataGenerator(audios, texts, input_length, label_length, nbatch, nfilt)
validation_generator = DataGenerator(audios_val, texts_val, input_length_val, label_length_val, nbatch, nfilt)

In [ ]:
# model constructor
train_model = brnn_ctc_lfbank(nfeature, nclass, lr_rate, momentum, True, False)

callbacks = [
    EarlyStopping(monitor="val_loss", patience=7, mode="min", restore_best_weights = True),
    ModelCheckpoint(filepath = weight_save_path,
                monitor = 'val_loss', mode = 'min', save_best_only=True, save_weights_only=True),
]

In [ ]:
# model train
history = train_model.ctc_model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers = 1,
                    epochs = epoch)

In [ ]:
# loss visualize
import matplotlib.pyplot as plt

figure = plt.figure(figsize = (10,3))
plt.plot(history.history["val_loss"], color = "red")
plt.plot(history.history["loss"])